<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2019


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2019/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>


<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> Handling data with Python </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b>September 16, 2019</b></p>

<hr style='height:5px;border:none' />

# Outline

**In the previous tutorial:**
* Basic unix commands
* Operators (number & string)
* Variables & lists
* Flow control (if, while)
* For loop

**In this tutorial:**
* Dictionary & JSON file
* Data frame
  * Reading CSV
  * Slicing, indexing
  * Updating
* Arrays
  * Slicing & indexing
* NIfTI format

# Dictionaries
<hr style="height:1px;border:none" />

## What is a dictionary? 

A **dictionary** is like a list. But unlike a list whose items are indexed by a number, we can refer items in a dictionary with keys. Here is an example of a dictionary.

In [1]:
bookInfo = {'Author':'Sweigart, Al',
            'Title':'Automate the Boring Stuff with Python',
            'Publisher':'No Starch Press',
            'Year': 2015,
            'Pages':503}

In [2]:
bookInfo

{'Author': 'Sweigart, Al',
 'Pages': 503,
 'Publisher': 'No Starch Press',
 'Title': 'Automate the Boring Stuff with Python',
 'Year': 2015}

Each item in the dictionary consists of a **key** and a **value** separated by a colon (`:`). You can use a key to access the value of the corresponding item. For example,

In [3]:
bookInfo['Author']

'Sweigart, Al'

In [4]:
bookInfo['Pages']

503

You can just get keys, to see what fields are in a dictionary.

In [5]:
bookInfo.keys()

dict_keys(['Year', 'Pages', 'Author', 'Publisher', 'Title'])

Likewise, you can just get values.

In [6]:
bookInfo.values()

dict_values([2015, 503, 'Sweigart, Al', 'No Starch Press', 'Automate the Boring Stuff with Python'])

You can also get a list of key-value pairs. This becomes handy when you are writing a for loop.

In [7]:
bookInfo.items()

dict_items([('Year', 2015), ('Pages', 503), ('Author', 'Sweigart, Al'), ('Publisher', 'No Starch Press'), ('Title', 'Automate the Boring Stuff with Python')])

So, *why should we care about dictionaries?* Sometimes it makes sense to organize multiple properties of an object / individual as a single entity, as opposed to a collection of separate variables. JSON file format is a universal format to describe dictionary-like information.

## JSON dictionary files (`.json`)

A JSON (JavaScript Object Notation) file contains pairs of keys and values, just as a dictionary in python. JSON files can be used to describe details about data sets, sessions, runs, tasks, or imaging parameters in an fMRI experiment. 

If you have mounted the flanker data from the previous class to `/tmp/Data/ds102` in your Docker image, then you can follow the following demonstration on JSON files.

In [8]:
import os
import json

dataDir = '/tmp/Data/ds102'

There are two JSON files in this directory: **`task-flanker_bold.json`** (describing the experiment design), and **`dataset_description.json`** (describing the data set). 

Let's examine what `task-flanker_bold.json` file looks like.

In [10]:
fJSON = os.path.join(dataDir,'task-flanker_bold.json')

The **`os.path.join`** function concatenates the path to a directory (in this particular case, `/tmp/Data/ds102` stored in variable `dataDir`) and a file name (in this case, `task-flanker_bold.json`). 

In [11]:
fJSON

'/tmp/Data/ds102/task-flanker_bold.json'

The file contains the following:

```
{
	"RepetitionTime": 2.0,
	"TaskName": "Flanker",
	"TaskDescription": "On each trial (inter-trial interval (ITI) varied between 8 s and 14 s; mean ITI=12 s),participants used one of two buttons on a response pad to indicate the direction of a central arrow in an array of 5 arrows. In congruent trials the flanking arrows pointed in the same direction as the central arrow (e.g., < < < < <), while in more demanding incongruent trials the flanking arrows pointed in the opposite direction (e.g., < < > < <). Subjects performed two 5-minute blocks, each containing 12 congruent and 12 incongruent trials, presented in a pseudorandom order.",
	"CogAtlasID": "tsk_4a57abb949a4f",
	"Manufacturer": "Siemens",
	"ManufacturerModelName": "Allegra",
	"MagneticFieldStrength": 3.0,
	"ScanningSequence": "Echo Planar",
	"MRAcquisitionType": "2D",
	"EchoTime": 0.030,
	"FlipAngle": 80.0
}
```

As you can see, there are different keys (`RepetitionTime`, `TaskName`, ..., `FlipAngle`) and the values associated with them.

To read this file as a dictionary object, we use the **`load()`** function under the **`json`** library.

In [15]:
g = open(fJSON,'r')
dataDic = json.load(g)
g.close()

We can see the entire dictionary object,

In [16]:
dataDic

{'CogAtlasID': 'tsk_4a57abb949a4f',
 'EchoTime': 0.03,
 'FlipAngle': 80.0,
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 3.0,
 'Manufacturer': 'Siemens',
 'ManufacturerModelName': 'Allegra',
 'RepetitionTime': 2.0,
 'ScanningSequence': 'Echo Planar',
 'TaskDescription': 'On each trial (inter-trial interval (ITI) varied between 8 s and 14 s; mean ITI=12 s),participants used one of two buttons on a response pad to indicate the direction of a central arrow in an array of 5 arrows. In congruent trials the flanking arrows pointed in the same direction as the central arrow (e.g., < < < < <), while in more demanding incongruent trials the flanking arrows pointed in the opposite direction (e.g., < < > < <). Subjects performed two 5-minute blocks, each containing 12 congruent and 12 incongruent trials, presented in a pseudorandom order.',
 'TaskName': 'Flanker'}

Or just a particular element.

In [17]:
dataDic['RepetitionTime']

2.0

One thing to note here is that a dictionary object does not have a particular ordering of elements, unlike lists. So the order of elements may not be the same as that of the original JSON file.

### Exercise
**Data set description**. Write a code to read the data set description JSON file **`dataset_description.json`** as a dictionary. Print out all the keys in the resulting dictionary.

# Data frames
<hr style="height:1px;border:none" />

Say, you have some data in the form of a table. For example, the file **`participants.tsv`** under the `ds102` directory contains the following information.

```
participant_id	gender	age
sub-01	F	21.94
sub-02	M	22.79
sub-03	M	19.65
sub-04	M	25.98
sub-05	M	23.24
sub-06	M	23.27
sub-07	D	34.72
sub-08	D	22.22
...
```

Here, columns represent different variables (`participant_id`, `gender`, and `age`), whereas rows represent different individuals / observations (`sub-01`, `sub-02`, and so on). We can handle this collection of information as a single **data frame** object. A Python library **Pandas** has a collection of utilities for data frames. So let's read this data as a data frame object.

[`<LoadTable.py>`](https://github.com/sathayas/fMRIClassFall2019/blob/master/HandlingData/LoadTable.py)

In [1]:
import os
import pandas as pd

Note that **`pandas`** library is imported with a short-hand notation **`pd`**. This becomes handy, as you do not have to type `pandas` every time you want to use functions in this library!

In [2]:
# data directory & file
dataDir = '/tmp/Data/ds102'
fileTable = os.path.join(dataDir,'participants.tsv')

# Loading the data file as a data frame object
ptData = pd.read_csv(fileTable, sep='\t')

The function **`read_csv`** reads the data table from a file (as specified in the variable `fileTable`). This function is designed for csv (comma-separated values) file format; since values are separated by tabs (or `\t`s), we need to specify a parameter **`set='\t'`**).

Here is what has been read from the file:

In [3]:
ptData

,participant_id,gender,age
0,sub-01,F,21.94
1,sub-02,M,22.79
2,sub-03,M,19.65
3,sub-04,M,25.98
4,sub-05,M,23.24
5,sub-06,M,23.27
6,sub-07,D,34.72
7,sub-08,D,22.22
8,sub-09,M,22.70
9,sub-10,D,25.24


## Useful methods for data frames

There are some useful methods associated with a data frame object. You can examine the first few observations in a data frame with the **`head()`** method.

In [4]:
ptData.head()

,participant_id,gender,age
0,sub-01,F,21.94
1,sub-02,M,22.79
2,sub-03,M,19.65
3,sub-04,M,25.98
4,sub-05,M,23.24


To get names of the columns, you can use **`columns`**.

In [6]:
ptData.columns

Index(['participant_id', 'gender', 'age'], dtype='object')

You can get some basic descriptive statistics for each column by using the **`describe()`** method.

In [7]:
ptData.describe()

,age
count,26.000000
mean,28.337692
std,7.779989
min,19.650000
25%,22.902500
50%,25.850000
75%,33.585000
max,49.790000


You can sort the entire data frame based on a variable using the **`sort_values()`** method. For example, we can sort the data according to the variable **`age`** in ascending order. I am showing only the 10 youngest subjects using the `head()` method.

In [11]:
ptData.sort_values(by='age').head(10)

,participant_id,gender,age
2,sub-03,M,19.65
10,sub-11,M,21.80
0,sub-01,F,21.94
7,sub-08,D,22.22
8,sub-09,M,22.70
1,sub-02,M,22.79
22,sub-23,M,22.79
4,sub-05,M,23.24
5,sub-06,M,23.27
17,sub-18,D,24.03


Or in the descending order by specifying **`ascending=False`**.

In [12]:
ptData.sort_values(by='age', ascending=False).head(10)

,participant_id,gender,age
13,sub-14,M,49.79
15,sub-16,M,49.03
16,sub-17,D,35.77
6,sub-07,D,34.72
24,sub-25,M,34.56
19,sub-20,M,34.19
14,sub-15,D,34.06
25,sub-26,M,32.16
23,sub-24,M,27.24
21,sub-22,D,27.00


## Indexing and slicing a data frame

In a data frame, you can access individual column (or variable) by

In [13]:
ptData.age

0     21.94
1     22.79
2     19.65
3     25.98
4     23.24
5     23.27
6     34.72
7     22.22
8     22.70
9     25.24
10    21.80
11    26.00
12    25.72
13    49.79
14    34.06
15    49.03
16    35.77
17    24.03
18    24.67
19    34.19
20    26.22
21    27.00
22    22.79
23    27.24
24    34.56
25    32.16
Name: age, dtype: float64

or

In [14]:
ptData['age']

0     21.94
1     22.79
2     19.65
3     25.98
4     23.24
5     23.27
6     34.72
7     22.22
8     22.70
9     25.24
10    21.80
11    26.00
12    25.72
13    49.79
14    34.06
15    49.03
16    35.77
17    24.03
18    24.67
19    34.19
20    26.22
21    27.00
22    22.79
23    27.24
24    34.56
25    32.16
Name: age, dtype: float64

You can slice rows in a data frame very easily. For example,

In [15]:
ptData[5:8]

,participant_id,gender,age
5,sub-06,M,23.27
6,sub-07,D,34.72
7,sub-08,D,22.22


You can slice rows with only a few selected variables only, using the **`loc`** method.

In [16]:
ptData.loc[5:8,['participant_id','age']]

,participant_id,age
5,sub-06,23.27
6,sub-07,34.72
7,sub-08,22.22
8,sub-09,22.70


And you can use the data itself to select a subset of data, like in arrays. For example, you can find subjects 30 years or older by

In [17]:
ptData[ptData.age>30]

,participant_id,gender,age
6,sub-07,D,34.72
13,sub-14,M,49.79
14,sub-15,D,34.06
15,sub-16,M,49.03
16,sub-17,D,35.77
19,sub-20,M,34.19
24,sub-25,M,34.56
25,sub-26,M,32.16


## Saving a data frame

You can manipulate a data frame and save it as a CSV file quite easily. Here is an example with the participant information data frame.

[`<SaveTable.py>`](https://github.com/sathayas/fMRIClassFall2019/blob/master/HandlingData/SaveTable.py)

In [21]:
# Sorting by age in descending order
ptData.sort_values(by='age', ascending=False, inplace=True)

First, the data frame was sorted by age in descending order. The parameter **`inplace=True`** specifies that the operation (sorting in this case) is applied to the data frame, resulting in an altered data frame.

In [22]:
ptData.head(10)

,participant_id,gender,age
13,sub-14,M,49.79
15,sub-16,M,49.03
16,sub-17,D,35.77
6,sub-07,D,34.72
24,sub-25,M,34.56
19,sub-20,M,34.19
14,sub-15,D,34.06
25,sub-26,M,32.16
23,sub-24,M,27.24
21,sub-22,D,27.00


From this data frame, we select observations with **`gender`** being **D** (I am not sure what it stands for). 

In [23]:
ptData[ptData.gender=='D']

,participant_id,gender,age
16,sub-17,D,35.77
6,sub-07,D,34.72
14,sub-15,D,34.06
21,sub-22,D,27.00
11,sub-12,D,26.00
9,sub-10,D,25.24
18,sub-19,D,24.67
17,sub-18,D,24.03
7,sub-08,D,22.22


We write this to a CSV file. To do so, we can use the **`to_csv`** method. The output file name is the first parameter (required). We also specify **`index=False`** to drop the index column (with sequential numbers in the first column). 

In [24]:
# output file name
fileOut = os.path.join(dataDir,'sorted_D.csv')

# selecting observations with gender=D, saving to a CSV file
ptData[ptData.gender=='D'].to_csv(fileOut, index=False)

This produces the output file **`sorted_D.csv`**, which looks like:

```
sub-17,D,35.77
sub-07,D,34.72
sub-15,D,34.06
sub-22,D,27.0
sub-12,D,26.0
sub-10,D,25.24
sub-19,D,24.67
sub-18,D,24.03
sub-08,D,22.22
```

### Exercise

**Sort by subject ID**. Load the file we just created, `sorted_D.csv`, into a data frame. Sort observations by their subject IDs. *Hint: you do not need to use `sep` parameter in the `read_csv` function.*